In [10]:
import pandas as pd

# Read x_train.txt
X = pd.read_csv('../data/x_train.txt', sep=' ', header=None)

# Read y_train.txt
y = pd.read_csv('../data/y_train.txt', sep=' ', header=None)


# Display the data
print("x:")
print(X.head())

print("\ny:")
print(y.head())


x:
        0         1         2         3         4         5         6    \
0 -2.619773 -2.619533 -1.199350 -1.083335 -1.000910 -0.366967 -2.164037   
1 -1.415579 -1.782544 -2.880270 -1.958863  1.159968  0.273030 -1.628728   
2 -2.745092 -1.382945 -1.626015 -1.282560 -0.663146  0.052349 -2.403322   
3  0.618998  0.455364 -0.115081  0.649040 -0.862207  2.308504  0.526114   
4 -0.070694 -0.550509 -0.565556 -0.693065 -0.573089 -0.395862  0.003170   

        7         8         9    ...        490        491        492  \
0 -1.210001 -0.658311 -1.489539  ...  10.849925  10.343346  10.717519   
1 -0.175813 -0.916857 -0.570166  ...  11.489417   5.195818   3.494627   
2 -0.765073 -0.394354 -0.806624  ...  13.934934   9.267515   4.705604   
3 -1.094852  1.088656 -0.481210  ...  12.021328   3.852231  11.059702   
4 -0.981609 -0.505775 -0.758430  ...   7.537788  11.229665  11.318915   

        493        494        495        496        497        498        499  
0  7.709295   5.894554  12.

In [11]:
#vars=[100, 101, 102, 103, 104, 105, 6]

In [12]:
#X = X[vars]

In [13]:
from supervised.automl import AutoML
import ray
from ray import tune

from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.search.bohb import TuneBOHB
from ray.train import report
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
import pandas as pd
import numpy as np
import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH
import ast

def train_ml_jar(columns):
    # Split data into training and validation sets
    vars = ast.literal_eval(columns)
    Xloc = X[vars]

    X_train, X_val, y_train, y_val = train_test_split(Xloc, y, test_size=0.2, random_state=42)

    # Define the model with the given configuration
    model = AutoML(mode="Compete",total_time_limit=3000,features_selection= False)
    
    # Train the model
    model.fit(X_train, y_train)

    # Predict on the validation set
    preds_proba = model.predict_proba(X_val)[:, 1]
    
    # Get the indices of the top 20% predictions
    top_20_percent_indices = np.argsort(preds_proba)[-int(0.2 * len(preds_proba)):]

    # Select the corresponding true labels and predicted labels for top 20%
    top_20_true = y_val.iloc[top_20_percent_indices].values
    top_20_preds = (preds_proba[top_20_percent_indices] >= 0.5).astype(int)
    
    # Calculate the number of correct class 1 predictions
    #correct_class_1_predictions = np.sum((top_20_true == 1) & (top_20_preds == 1))
    #correct_class_1_predictions = np.sum(np.logical_and(top_20_true == 1, top_20_preds == 1))
    correct_class_1_predictions = sum(1 for true, pred in zip(top_20_true, top_20_preds) if true == 1 and pred == 1)

    print(correct_class_1_predictions, len(X_val))
    
    # Calculate the number of features used
    num_features_used = Xloc.shape[1]

    #scaled score calculation
    customer_scaled = (correct_class_1_predictions / len(top_20_preds)) * 1000
    customer_gain = 10 * customer_scaled
    variable_cost = 200 * num_features_used
    custom_score = customer_gain - variable_cost
    
    # Compute the custom score
    #custom_score = 10 * correct_class_1_predictions - 200 * num_features_used
    report({
        "custom_score": custom_score, 
        "correct_class_1_predictions": correct_class_1_predictions, 
        "num_features_used": num_features_used
    })
    return custom_score



 
columns = ['[101, 102, 103, 105]', '[101, 102, 103]', '[101, 102, 105]', '[101, 103, 105]', '[102, 103, 105]', '[104, 102, 103, 105]', '[100, 101, 102, 103, 105]', '[100, 101, 102, 103]', '[100, 101, 102, 105]', '[100, 101, 103, 105]', '[100, 102, 103, 105]', '[100, 104, 102, 103, 105]', '[100, 101, 102, 103, 104, 105]']

results = {}
for column_set in columns:
    results[column_set] = train_ml_jar(column_set)
    print(f"{column_set} : {results[column_set]}")


AutoML directory: AutoML_1
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
'module' object is not callable
1_DecisionTree logloss 0.705761 trained in 3.94 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 5-fold CV Shuffle,Stratify
* Step simple_algorithms will try to check up to 4 models
'module' object is not callable
1_DecisionTree logloss 0.702216 trained in 2.81 seconds
'module' object is not callable
2_DecisionTree logloss 0.734137 traine

In [14]:
print(results)

{'[101, 102, 103, 105]': 7150.0, '[101, 102, 103]': 7200.0, '[101, 102, 105]': 7200.0, '[101, 103, 105]': 7150.0, '[102, 103, 105]': 7100.0, '[104, 102, 103, 105]': 6900.0, '[100, 101, 102, 103, 105]': 7050.0, '[100, 101, 102, 103]': 7000.0, '[100, 101, 102, 105]': 7150.0, '[100, 101, 103, 105]': 6750.0, '[100, 102, 103, 105]': 7300.0, '[100, 104, 102, 103, 105]': 6900.0, '[100, 101, 102, 103, 104, 105]': 6950.0}


In [17]:
max(results.values())

7300.0